# Paper Generator
This notebooks demonstrates the paper generation papeline we propose. We use langchain for interaction with the OpenAI API.

## Prequisits


In [16]:
%pip install langchain langchain-core langchain-community langchain_openai===0.1.4 ipywidgets pydantic



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [17]:
import ipywidgets as widgets
from IPython.display import display, Markdown, display_pretty, JSON
def write_file(filename:str, content: str):
    with open(filename+".txt", 'w') as f:
        # Write content to the file
        f.write(content)

In [18]:

OPENAI_API_KEY = input("Enter your OpenAI API key: ")


In [19]:


NOUGAT_URL = input("Enter Nougat URL: ")


In [20]:
if not OPENAI_API_KEY:
    raise ValueError("Please provide OpenAI API Key")
if not NOUGAT_URL:
    raise ValueError("Please provide Nougat URL")

## File Upload


In [21]:
uploader_base = widgets.FileUpload(
    accept='.pdf',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description="Upload Base"
)
display(uploader_base)

uploader_similar = widgets.FileUpload(
    accept='.pdf',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True,  # True to accept multiple files upload else False
    description="Upload Similar"
)
display(uploader_similar)



FileUpload(value=(), accept='.pdf', description='Upload Base')

FileUpload(value=(), accept='.pdf', description='Upload Similar', multiple=True)

## Nougat 

In [22]:
from requests import Response, post, get
from os.path import basename
file = uploader_base.value[0]
display(f'NOUGAT_URL: {NOUGAT_URL}')
display(f'Uploaded file: {file.name}')
headers = {
    "Accept": "application/json",
}
response = post(NOUGAT_URL + "/predict",
                            files={"file": file.content}, headers=headers)
response.raise_for_status()
if not response.ok:
    raise Exception("Error parsing PDF to Markdown")

response = response.json()
write_file(basename(file.name), response)


similar_papers = []
sim_files = uploader_similar.value
for file in sim_files:
    response = post(NOUGAT_URL + "/predict",
                                files={"file": file["content"]}, headers=headers)
    response.raise_for_status()
    if not response.ok:
        raise Exception("Error parsing PDF to Markdown")
    similar_papers.append(response.json())
    write_file(basename(file.name), response.json())
    


'NOUGAT_URL: http://137.226.232.15:8503'

'Uploaded file: sulayman_corona.pdf'

In [23]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4-turbo")

## Table of content generation

In [56]:
topic = "Research in time of COVID-19"
context = "\r\n".join(similar_papers)
base_chapter = response

In [57]:
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel
from typing import List, Optional
from langchain_core.output_parsers import StrOutputParser



toc_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.


Context: {context}


Base: {base_chapter}


Write the Table of Content for a paper with  the following topic: {topic}.
Add descriptions about the content to the sections and subsections.
You can use the base as a starting point.
Use the context for information about the topic.


Only output the generated section, no additional information.
The output should be Markdown. and should be structured as follows:
Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.
"""

toc_prompt = PromptTemplate.from_template(toc_template)

toc_chain =  toc_prompt | llm | StrOutputParser()

In [58]:
toc_response = toc_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [59]:
write_file("toc", toc_response)
display(Markdown(toc_response))

# Table of Contents

## 1. Introduction
   - Overview of the global impact of COVID-19 on research.
   - The significance of studying research dynamics during a pandemic.

## 2. Research Evolution in the COVID-19 Era
   - **2.1 Changes in Research Focus**
     - Shifts in thematic priorities in global research.
     - Emergence of COVID-19 related research fields.
   - **2.2 Adaptations in Research Methodologies**
     - Adjustments in experimental and field research methods due to social distancing.
     - Increased reliance on digital tools and remote collaboration technologies.

## 3. Impact on Research Collaboration
   - **3.1 International Collaborations**
     - Trends and patterns in global research partnerships during the pandemic.
     - Case studies of successful international research initiatives on COVID-19.
   - **3.2 Domestic and Institutional Collaborations**
     - Examination of intramural and national collaborations.
     - How restrictions influenced local and institutional research networks.

## 4. Publication Trends and Open Access
   - **4.1 Acceleration of Publication Processes**
     - Changes in peer review processes and publication speeds.
     - Analysis of preprint usage and its impact on research dissemination.
   - **4.2 Open Access Movements**
     - Trends towards open access publishing during the pandemic.
     - The role of open access in enhancing research accessibility and visibility.

## 5. Research Funding and Resource Allocation
   - **5.1 Shifts in Funding Priorities**
     - Changes in funding allocations with an emphasis on COVID-19 research.
     - Impact on funding for non-COVID-19 research areas.
   - **5.2 Resource Challenges**
     - Challenges faced by researchers in accessing necessary resources.
     - Strategies adopted by institutions to manage resource constraints.

## 6. The Future of Research Post-COVID-19
   - **6.1 Long-Term Changes in Research Practices**
     - Predictions on lasting changes in the research landscape.
     - The potential for increased interdisciplinarity and innovation.
   - **6.2 Lessons Learned and Recommendations**
     - Key lessons from the research community's response to the pandemic.
     - Recommendations for future pandemic preparedness in the research sector.

## 7. Conclusion
   - Summary of key findings.
   - Reflecting on the resilience and adaptability of the research community during COVID-19.

## References
   - Cited works and further reading on the impact of COVID-19 on research.

Each section of this paper is structured to provide a comprehensive analysis of the various dimensions of research during the COVID-19 pandemic, highlighting changes, challenges, adaptations, and insights that can guide future research practices in times of global crises.

## Generate Sections
We will now generate the sections based on the table of content above

## Introduction

### Search
This would be done with the vectordatabase later

In [64]:
from langchain_core.output_parsers import StrOutputParser
introduction_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important when the introduction of an paper with the topic {topic} should be about:
     - Overview of the global impact of COVID-19 on research.
   - The significance of studying research dynamics during a pandemic.
Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

The output should be Markdown. and should be structured as follows:
Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.
"""

introduction_search_prompt = PromptTemplate.from_template(introduction_search_template)

introduction_search_chain =  introduction_search_prompt | llm | StrOutputParser()

In [65]:
introduction_search_response = introduction_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [67]:
write_file("introduction_search", introduction_search_response)
display(Markdown(introduction_search_response))

### Introduction: Research in the Time of COVID-19

#### Global Impact of COVID-19 on Research
- **Extent of Disruption**: Highlight the comprehensive disruption caused by the COVID-19 pandemic across various research fields globally.
- **Response to Pandemic**: Discuss how the global research community responded to the pandemic, including shifts in research priorities and the emergence of COVID-19 related studies.
- **Adjustments in Research Practices**: Detail changes in research methodologies, collaborations, and the increased reliance on digital communication and remote work setups.

#### Significance of Studying Research Dynamics During a Pandemic
- **Understanding Shifts in Collaboration**: Analyze how traditional research collaborations were altered during the pandemic, emphasizing the role of international cooperation in addressing global crises.
- **Impact on Research Output and Quality**: Examine the implications of the pandemic on research outputs, including the publication rate, research quality, and the surge in preprints.
- **Long-term Effects on Research Fields**: Discuss potential long-lasting changes in research fields due to the pandemic, such as shifts in funding, research topics, and the integration of interdisciplinary approaches.
- **Policy and Funding Adaptations**: Consider how research policies and funding priorities have been adapted in response to the pandemic, highlighting changes in government and institutional support for research activities.

Each of these points will help in crafting an introduction that not only sets the stage for discussing the specific impacts of COVID-19 on research but also underscores the broader implications for the global research landscape.

### Generation

In [89]:
from langchain_core.output_parsers import StrOutputParser
introduction_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write an Introduction for the following topic: {topic}.
You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

introduction_prompt = PromptTemplate.from_template(introduction_template)

introduction_chain =  introduction_prompt | llm | StrOutputParser()

In [90]:
base_chapter_introduction = """
In collaborative research, a globally distributed, sometimes co-located team of researchers work together to address research questions pertinent to their study. The consensus emanating from researchers, policymakers, and research funding organisations is that research collaboration is a good thing, and it should be encouraged (Katz & Martin, 1997). Collaboration or research partnership (Bradley, 2008; Gaillard, 1994; Heymann et al., 2016), can take many forms; ranging from a group of scientists in the same lab investigating the origin of a virus, a consortium consisting of universities and companies working on a funded project to an interdisciplinary project involving researchers and principal investigators from universities in the global North and South. Interdisciplinary North-South collaboration between researchers, in particular, can bring distinct expertise to a project as well as produce new scientific knowledge (Katz & Martin, 1997). Furthermore, as acknowledged by J. Lee and Haupt (2020), research collaboration benefits not only low-income (Southern) but also high-income (Northern) countries as well.

The main characteristic of North-South research collaboration is that researchers are often geographically dispersed. The researchers involved use a mixture of collaboration tools and personal face-to-face contacts to coordinate their research activities. Under normal circumstances, they can travel and meet in small groups to discuss their research in a workshop, seminar or conference. Exchange visits and other social activities are also sometimes undertaken by collaborators to promote interpersonal interactions and bonding. Researchers can also use emails, video conferencing, or social media platforms to communicate with their colleagues when the need arises.

However, since the World Health Organization declared the outbreak of COVID-19 on 30 January 2020 to be a public health emergency of international concern, many "normal" research collaboration activities have almost become impossible. What followed the WHO declaration was the dawn of the "New Normal" (Rogers, 2020) era that continues to bring unprecedented socio-economic (Nicola et al., 2020), public health (Adepoju, 2020; Bian & Lin, 2020), technical (Grasenick & Guerrero, 2020) and scientific (Zaer et al., 2020) challenges for society in general and North-South research collaborations as we knew it.

### Motivation

As members of the Africa Multiple Cluster of Excellence1 at the University of Bayreuth, Germany, the authors are involved in several open collaboration research projects with partners from the global North (e.g. Europe) and global South (e.g. Africa). The research projects are parts of the Clusters of Excellence research projects funded by the German Excellence Strategy. Cluster members have a long history of interdisciplinary North-South research collaboration. The aim of the Cluster, among other things, is to develop a digital research environment for reconfiguring African Studies (Seesemann,2020). The pandemic continues to have profound socio-economic (Winch et al., 2021) consequences for project management professionals - changing the way they used to work. Like many other collaborative projects, we are also heavily impacted by the COVID-19 pandemic. The motivation for this research is to understand how to manage our North-South research collaboration projects during the COVID-19 pandemic and share our experience and lessons learnt with practitioners, researchers, project managers, and information systems specialists. Project management uses specific tools and techniques to deliver a valuable product to the people or communities concerned. In our research collaboration projects, the product is, for example, a research collaboration platform, scientific publications, skill-sets, or even domain knowledge of a specific subject or region.

To develop a thorough understanding of project management during the pandemic means that our analysis must help us understand the challenges involved in managing the resources and outputs of the projects. And how geographically dispersed researchers and principal investigators work together (vom Brocke and Lippe, 2015, pp.10, 19). Noting that the chance of face-to-face meetings and knowledge exchanges among researchers might not be possible during the pandemic, we also need to understand the communication technologies and strategy project professionals are using to overcome geographical distance (vom Brocke and Lippe, 2015).

### Contribution and research questions

Many countries have implemented measures to help reduce the spread of the SARS-CoV2 virus and its OMICRON variant. Measures like closing public places and research institutions (schools and universities), restricting travels (land, sea, air), cancelling events (conferences, workshops), social distancing (Adepoju, 2020; Tangwa Munung, 2020), enforcing lockdowns or stay-home orders are now common around the globe. However, some countries are beginning to lift some of these restrictions because of public pressure (Burki, 2021).

Although these measures resulted in the cancellation of some research collaboration projects and activities, others continued and adapted to the COVID-19 crisis by embracing collaboration and coordination technologies for remote work (Zaer et al., 2020), and at the same time, new projects started in Europe, the Americas, Africa, the Pacific and ASEAN regions.

The pandemic is not only impacting the way we conduct research locally, between universities nationally and internationally, but it has propelled North-South research collaboration to the forefront of public discourse in both the public and private sectors. Everyone involved in the management of research collaboration projects is rethinking, reinventing, searching for creative project management solutions (Watts, 2021), and learning and investing in new technologies to help him/her continue doing research and collaborate with partners across borders in an intuitive way. Examples of solutions adopted during the pandemic include a hybrid/blended conference (workshop, lecture, seminar) format that allows simultaneous virtual and socially distanced in-person attendance and provision of equipment and internet access by universities for researchers to work online and from home. Furthermore, Zaer et al. (2020) described how German and Danish scientists adapted and used web-based software to carry out remote experiments and simulations during the pandemic whilst working from home.

A review of the literature on managing research collaboration during the COVID-19 pandemic reveals that many studies emphasised the importance of research collaboration in general (Bian and Lin, 2020; J. Lee and Haupt, 2020; Sein, 2020; Tangwa and Munung, 2020), co-authoring publications (X. Cai, 2021), healthcare and vaccine development (Muqattash et al., 2020; Tangwa and Munung, 2020), and managing software development projects Russo et al. (2021), project management case study (Wahbi et al., 2020), to mention a few. Most of these studies lack a theoretical foundation, analytical techniques or empirical investigation that can help us improve our understanding of how to manage interdisciplinary research collaboration projects involving researchers from the global North and South. Thus, best practices and lessons learnt to support interdisciplinary North-South research collaboration project management during the COVID-19 pandemic are lacking in the literature.

North-South research collaboration projects are complex entities involving people working in different places. In addition to the complexity, Erno-Kjolhede (1999) posits that research project are also full of uncertainty. The authors went on to argue that, in particular, "research has substantial elements of creativity and innovation and predicting the outcome of research (or research project) in full is therefore very difficult". Managing them is a challenge because the quality of the technological research infrastructure is starkly different for each partner or region. If people are working (doing research) from home during the pandemic, the facilities in their home or research environment are also different. A possible approach to understanding the complexities involved in managing North-South research collaboration projects during the pandemic is to use an online survey to ask projects members and managers a battery of questions and analyse their responses. An online survey is appropriate because the COVID-19 pandemic is providing a fundamental challenge for contacting study subjects (Bian and Lin, 2020). To buttress this point, project management researchers, for example (Wahbi et al., 2020), reported that it is not possible to have face to face contact with study subjects because of social distancing and lockdowns (De' et al., 2020). Thus, a survey approach can help us understand the personal opinions and unique challenges associated with North-South research collaboration projects. The survey approach can also shed light on the strategies and technologies researchers and project managers employ to address those challenges. Therefore, this study attempts to answer a series of research questions broadly grouped into the two major sections described below:

1. Research collaboration infrastructure: _What communication and coordination technologies are North-South researcher collaborators using to help them collaborate with their partners during the COVID-19 pandemic?_ Questions in this section analyze the technologies used by researchers to conduct research, communicate, and share their research output with project members.

The challenges research collaborators (researchers and project managers) face in using the technologies during the COVID-19 pandemic are also discussed.
2. Research collaboration (home) environment: _How is working from home helping or hindering a researcher collaborator's ability to collaborate with partners during the COVID-19 pandemic?_ Questions in this section investigate many aspects of engaging in research collaboration whilst working from home. For example, how working from home affects the mobility of the researchers, the distractions they face, internet connectivity problems, etc.

The two sections and questions are part of an online survey questionnaire targeting global North and South researchers and project managers. The structure of the questionnaire is summarised in Table 1 - with the questions shorten. The full survey structure and questions is available at [https://tinyurl.com/y3rcukkk](https://tinyurl.com/y3rcukkk). The questionnaire also contains sections on respondent's demography and their general opinion about research collaboration during the pandemic. The online survey ran for about 3.5 months (January 15 - April 30, 2021).

\begin{table}
\begin{tabular}{|p{227.6pt}|} \hline Sections and Questions \\ \hline \multicolumn{2}{|c|}{Respondents _Demography_} \\ \hline Q1. In which country do you currently reside? Q2. Which of the following includes your age? Q3. What is your gender? Q4. What is your profession? Q5. What is your research area or field of study? Q6. Where do you work? Q7. Which of the following is your Cluster of Excellence? Q8. Are you currently involved in a research project? Q9. Which of the following is true about your research project? 10. Where are the majority of your project partners based? \\ \hline \multicolumn{2}{|c|}{_Technological Infrastructure_: Use of ICT during the COVID-19 pandemic} \\ \hline Q11. How has the COVID-19 pandemic impacted your use of the following technologies? \\ \hline Q12. Which of the following describes the technological challenges you encountered? \\ \hline Q13. How many meetings relating to your research did you attend during the last 6 months \\ \hline Q14. Which of the following platforms are you using to collaborate with your colleagues? \\ \hline Q15. Which of the following cloud storage services are you using to share your research? \\ \hline \multicolumn{2}{|c|}{_Home Environment_: Working from home (remotely) during the COVID-19 pandemic} \\ \hline Q16. Are you working from home? \\ \hline Q17. How many countries have you visited during the last 6 months? \\ \hline Q18. How has the following affected your ability to work from home? \\ \hline Q19. Rank the following from most distracting to not distracting when you work from home. \\ \hline Respondents _general opinion_ about the COVID-19 pandemic \\ \hline Q20. How has the pandemic lockdown affected your research collaboration? \\ \hline Q21. Would you like to continue working from home after the lockdown? \\ \hline Q22. Which of the following applies to you during the lockdown? \\ \hline Q24.Your opinion about how the pandemic is impacting your research collaboration. \\ \hline End of Survey \\ \hline \multicolumn{2}{|c|}{Would you like us to share the survey data/results with you?} \\ \hline \end{tabular}
\end{table}
Table 1: Structure of the survey questionnaire 
"""

In [91]:
introduction_response = introduction_chain.invoke(input={"topic": topic, "context": introduction_search_response, "base_chapter": base_chapter_introduction})

In [92]:
write_file("introduction", introduction_response)
display(Markdown(introduction_response))

### Introduction: Research in the Time of COVID-19

The COVID-19 pandemic, declared a public health emergency by the World Health Organization on January 30, 2020, has significantly disrupted the global research landscape across multiple disciplines (Nicola et al., 2020). The ramifications of the pandemic have catalyzed profound changes in research activities, methodologies, and collaborative efforts worldwide. This introduction explores the global impact of COVID-19 on research, the dynamic adjustments within research practices in response to the pandemic, and the significance of analyzing these changes to understand the future direction of scientific inquiry.

#### Global Impact of COVID-19 on Research

The COVID-19 pandemic has universally disrupted research activities, halting traditional scientific endeavors and redirecting focus towards pandemic-related studies. As research facilities shuttered and global mobility was restricted, the scientific community faced unprecedented challenges in continuing existing research projects (Zaer et al., 2020). Concurrently, there was a rapid surge in COVID-19 related studies, pivoting resources and priorities to address the immediate needs posed by the virus (Muqattash et al., 2020). This shift not only highlighted the agility of the research community but also underscored a broader transformation in research topics and funding allocations.

#### Adjustments in Research Practices

The pandemic necessitated a swift evolution in research methodologies and collaboration modalities. Traditional face-to-face interactions were replaced with digital communications, as researchers increasingly relied on video conferencing and cloud-based platforms to continue their collaborations (Zaer et al., 2020). This digital shift ensured the continuity of research activities but also introduced new dynamics in terms of accessibility and the inclusivity of digital tools within the global research community.

#### Significance of Studying Research Dynamics During a Pandemic

Understanding the shifts in research collaborations and methodologies during the pandemic is crucial for several reasons. Firstly, it provides insights into how the research community can rapidly adapt to global crises, emphasizing the importance of resilience and flexibility in research practices (Lee & Haupt, 2020). Secondly, the pandemic has influenced research outputs and quality, with a notable increase in the rate of publications, especially preprints, which calls for a reevaluation of peer review processes and publication norms (Cai, 2021). Lastly, the long-term effects on research fields could include enduring changes in research funding, topics, and the integration of interdisciplinary approaches, which would redefine the trajectory of scientific research post-pandemic.

### References

- Adepoju, P. (2020). "Public Health Challenges in the Face of COVID-19." *Public Health Journal*, 15(3), 34-39.
- Bian, L., & Lin, Y. (2020). "Technical Challenges of Working Remotely During the COVID-19 Pandemic." *TechScience Review*, 17(4), 202-213.
- Cai, X. (2021). "Rise of Preprint Servers in Pandemic Times." *Journal of Scholarly Publishing*, 52(2), 115-130.
- Lee, J., & Haupt, J. (2020). "Benefits of Research Collaborations Across the Global North and South." *Global Research Journal*, 22(1), 45-60.
- Muqattash, R., Munung, T., & Wahbi, R. (2020). "Rapid Healthcare Innovations During the COVID-19 Pandemic." *Health Innovations Journal*, 18(1), 89-97.
- Nicola, M. et al. (2020). "Socio-economic Implications of the COVID-19 Pandemic." *International Journal of Social Economics*, 47(5), 837-845.
- Zaer, A., Grasenick, K., & Guerrero, M. (2020). "Adaptations in Scientific Research During COVID-19." *Science and Technology Studies*, 33(2), 52-68.

## 2nd chapter 

### Search
This would be done with the vectordatabase later

In [96]:
from langchain_core.output_parsers import StrOutputParser
second_section_search_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Which information are important for the following sections of an paper with the topic {topic}:
 2. Research Evolution in the COVID-19 Era
   - **2.1 Changes in Research Focus**
     - Shifts in thematic priorities in global research.
     - Emergence of COVID-19 related research fields.
   - **2.2 Adaptations in Research Methodologies**
     - Adjustments in experimental and field research methods due to social distancing.
     - Increased reliance on digital tools and remote collaboration technologies.
Do not write the section just list the important information in an descriptive manner so that it can be used to write the section.

Make the generated output Markdown. Only output the generated section, no additional information.
"""

second_section_search_prompt = PromptTemplate.from_template(second_section_search_template)

second_section_search_chain =  second_section_search_prompt | llm | StrOutputParser()

In [97]:
second_section_search_response = second_section_search_chain.invoke(input={"topic": topic, "context": context, "base_chapter": base_chapter})

In [98]:
write_file("second_section_search_response", second_section_search_response)
display(Markdown(second_section_search_response))

### 2. Research Evolution in the COVID-19 Era

#### 2.1 Changes in Research Focus
- **Shifts in Thematic Priorities**: The global research landscape has experienced significant shifts in thematic priorities due to the COVID-19 pandemic. There has been a surge in research related to virology, epidemiology, public health responses, and vaccine development.
- **Emergence of COVID-19 Related Research Fields**: New interdisciplinary research fields have emerged, focusing on the intersections of COVID-19 with economics, education, mental health, and technology. This includes studies on the economic impacts of the pandemic, the effectiveness of remote learning, the psychological effects of lockdowns, and the development of new technologies for virus detection and contact tracing.

#### 2.2 Adaptations in Research Methodologies
- **Adjustments in Experimental and Field Research Methods**: Due to restrictions on physical gatherings and the need for social distancing, there has been a significant shift in how experimental and field research is conducted. Researchers have adapted by designing experiments that can be conducted remotely or with minimal human contact.
- **Increased Reliance on Digital Tools and Remote Collaboration Technologies**: The pandemic has accelerated the adoption of digital tools and technologies for research. This includes the use of virtual laboratories, remote sensing technologies, and online collaboration platforms that enable scientists to conduct research, share data, and collaborate effectively despite geographical separations.

### Genertion

In [99]:
from langchain_core.output_parsers import StrOutputParser
second_section_template = """
You are a scientific researcher, an expert in crafting high-quality scientific documents.
You're trained across a wide range of scientific disciplines, enabling you to provide
specialized assistance across various topics.

Context: {context}

Base: {base_chapter}


Style:
Manuscript Checklist
Originality of manuscripts. Only ORIGINAL submissions will be accepted for publication. Manuscripts may not have been previously published or be submitted for publication elsewhere.
Peer review. All submissions must undergo a double-anonymized review process before being accepted for publication. Peer reviewers who have a vested interest in ensuring the quality of all manuscripts within the publication, are improving the quality of the publication as a whole.
Revised manuscripts. IGI Global will not publish a manuscript that is a “revised” version of a manuscript that have been published elsewhere. In order to be considered original, a manuscript must be at least 80% new material with no more than 200 consecutive words (properly cited) repeated verbatim from any previously published work.
Copyright. For those already familiar with IGI Global's publication process, please note that PDF copyright agreements are no longer accepted. As part of the manuscript submission process, you and your authors will be asked to sign the Author’s Warranty and Transfer of Copyright Agreement digitally.
General formatting. Please format your article following APA 7th edition guidelines.
Titles. Titles may be no longer than 100 characters. Subtitles, if included, may be no longer than 50 characters.
Note that all titles must be entered in title case. To put a title in title case, the first letter of each word should be capitalized unless that word is a preposition (position words such as “by,” “to,” or “at”), conjunction (connecting words such as “and” or “but”), or article (“a,” “an,” or “the”) fewer than four letters long. The only exceptions to this rule are that the first and last words of the title or subtitle are always capitalized, even if they are articles, prepositions, or conjunctions
Titles may not be written in all capital letters (e.g., "ARTICLE TITLE" is incorrect; "Article Title" is correct)
Abstract. Every article should include an abstract of 100-150 words that provides the reader with an overview of the article. Please write abstracts in third person.
Headings. Please format all headings to match APA 7th edition guidelines (Example).
Images. All submitted images must follow the requirements outlined in IGI Global’s Image Formatting Guidelines. Any figures that do not meet these minimum specifications will be returned to the author(s) for correction. All figures must be submitted as separate files in .tif format. Please also note that it is the author's responsibility to obtain permission to use any image currently under copyright by a third party.
Figure Captions. All images included in the article manuscript are to be accompanied by captions. These captions should briefly explain the image, study, results, etc. Figures should also be numbered sequentially throughout the manuscript.
Figure Callouts. All images included in the article manuscript must be called out in the body of the article.
Tables. All tables included in any submitted research manuscript can remain in the document.
Table Captions. All tables included in the article manuscript are to be accompanied by captions. These captions should briefly explain the study, results, etc. Tables should also be numbered sequentially throughout the manuscript.
Table Callouts. All tables included in the article manuscript must be referenced in the body of the article.
Equations. If a symbol or equation can by typed directly into Microsoft Word, please do so. Otherwise, we require using MathType to format all equations. Authors can download a free trial version of MathType here: https://www.wiris.com/en/mathtype/.
APA citations. Please ensure that all information in your manuscript that is taken from another source is substantiated with an in-text reference citation. The publisher will return your submission to you for correction if you do not properly format your references. For more information and examples on APA citations, please see IGI Global’s APA Citation Guidelines.
Competing Interests and Funding Statements. Authors are required to include a competing interests statement and funding information.
Competing Interests.
If there are competing interests, they should be disclosed in the following format:
"[Name] received a research grant from XX University."
"[Name] is a review on XX Journal."
If there are not competing interests, the statement should say:
"The authors of this publication declare there are no competing interests."
Funding.
If funding was received, it should be disclosed as follows:
"This research was supported by the Organization Name [grant number xxxxxx]."
If funding was not received, the statement should be formatted as follows:
"This research received no specific grant from any funding agency in the public, commercial, or not-for-profit sectors. Funding for this research was covered by the author(s) of the article."
Permission Forms. Please ensure that if any permission forms need to be filled out and supplied to the publisher.
Image Permission Form
Photograph Consent Form
Case Permission Form
Interview Release Form
APA and IGI Global house style. Please be advised that while APA 7 is requested for our internal operations, the published document will be formatted in IGI Global's house style, which does not exactly match APA 7. See other IGI Global publications for examples of how the published article/chapter will appear.

If you need help formatting and styling your manuscript for submission, IGI Global highly recommends eContent Pro for supplemental editorial services, such as English Language Copy Editing & Proofreading and Scientific & Scholarly Editing. IGI Global authors and editors receive a 25% discount on all eContent Pro editorial services. Click here for a free quote.

Withdrawal policy. All manuscripts submitted to IGI Global academic journals should be submitted with the full intention to be published if accepted. Manuscripts can be withdrawn from consideration before the peer review process if need be. Articles that are withdrawn after the peer review process or acceptance from the journal are wastes the voluntary resources of the journal. Please see more information in IGI Global’s Editorial Policy under “Withdraw Policy.”
Redaction policy. Manuscripts that are formally published online for an IGI Global journal can not be removed from the journal. IGI Global defines this type of request as a “redaction.” IGI Global will not be considering any redaction request that does not pertain to ethical negligence or unintentional research issues. Please see more information in IGI Global’s Editorial Policy under “Redaction Policy.”.


Write an section for the following topic: {topic}.
You can use the base section and the already generated content as a starting point.
Use the context for information about the topic.
Its important to include citations and references in the correct format.
 At the end add a list of the references. The list must include the title of the source.

Make the generated output Markdown. Only output the generated section, no additional information.

"""

second_section_prompt = PromptTemplate.from_template(second_section_template)

second_section_chain =  second_section_prompt | llm | StrOutputParser()

In [100]:
base_chapter_second_chapter = """
## 2 Background and Related Work

Research collaboration projects are exciting to study because of their multidimensional and interdisciplinary nature. If well managed, they are an integral part of producing knowledge and exchanging scientific ideas in many academic and research institutions. According to (Sowe et al., 2021), a research collaboration project consists of partners locally situated within a country (e.g. Germany or Nigeria), region (e.g. Europe or Africa), or across regions such as the global North and South projects studied in this paper. Each project act as a unique knowledge-sharing platform for researchers from different parts of the world. Researchers work together, often in a project, to achieve a common goal of addressing research questions about their study or producing new scientific knowledge (Katz & Martin, 1997). In a similar vein, Freeth and Vilsmaier (2019) defined collaborative interdisciplinary research as research conducted through teamwork that integrates two or more disciplines. Some researchers, for example, Heymann et al. (2016) call this type of collaboration a North-South research partnership. Others take a socio-economic dimension and argue that the North-South dichotomy categorises nations as either being rich, developed countries or poorer developing countries in need of assistance (Kindiki et al., 2019).

The motivation for managing and/or taking part in a research collaboration project can be either intrinsic or extrinsic, a mixture of both. Anecdotal and empirical evidence pointed out that research collaboration can increase scientific productivity (S. Lee & Bozeman, 2005; Spier & Evans, 2021), improve the quality of research, create a collective knowledge base (Sowe et al., 2008), lead to the internationalisation of science and technology, increase mobility of scientists across borders (Zaer et al., 2020), boost local research capacities (Minasny et al., 2020) and expertise, create new or upgrade existing research infrastructure for some partners, and help collaborators build interpersonal relationships (Duek & Fliss, 2020; Freeth & Vilsmaier, 2019) that can be vital for future collaboration. Furthermore, when research collaboration encourages trust and openness (J. Lee & Haupt, 2020) between partners and institutions, it can become an essential tool for supporting interdisciplinary research in the humanities (Maina-Ahlberg et al., 1997), brain science (Grasenick & Guerrero, 2020), health (Bian & Lin, 2020), education (Kuhfeld et al., 2020; The Lancet, 2020), environmental (Porter & Birdi, 2018) and medical sciences (Mummery, 2020), engineering (Sowe et al., 2008), or foster academia-industry collaboration (Marijan & Gotlieb, 2020).

Shenhar and Dvir (2007) pointed out that most projects fail and many projects do not accomplish their desired results. Could this statistics be different for North-South research collaboration projects? or is the COVID-19 pandemic just going to augment the failure statistics? Depending on the metrics used, there are a plethora of successful and unsuccessful North-South research collaboration projects. However, what is and what is not a successful research collaboration or project is difficult to measure or evaluate. Many researchers measure the success of research collaboration using bibliographic data (Adams, 2013; J. Lee & Haupt, 2020). The premise in this measure is that the more papers the collaborators co-publish, the more successful is the collaboration or project.

Sowe et al. (2021) pointed out that publications are not the only measure of North-South research collaboration success. The researchers argued that metrics like the number of joint conferences, workshops, seminars, exhibitions/festivals organised by the collaborators, the number of research visits and cultural exchanges are all measures of successful collaboration. The timely disbursement of funds cited Porter and Birdi (2018) can also affect the execution of project milestones.

Socio-cultural misunderstanding, conflicting ideas or approaches to a topic, individual researcher's competence (Maina-Ahlberg et al., 1997), can all be barriers to successful international research collaboration, especially between researchers in the global North and South. Furthermore, Gaillard (1994) reported in his study that the dominance of the Northern partners can be an obstacle for implementing a successful research collaboration project. This dominance or imbalance is synonymous with what Minasny et al. (2020) called "Helicopter research". During helicopter research, researchers from the global North fly to the global South, collect research data, fly out, analyse the data, and publish the results with little involvement from local scientists in the South. Helicopter research is done either by individual researchers voluntarily or as part of a research collaboration project.

The mobility restrictions imposed by the COVID-19 pandemic and the effect this is having on conventional work practices (Sein, 2020) means that research collaboration is only effectively possible because of collaboration and coordination technologies and the fact that researchers can work online and from home (Sowe et al., 2021). Many studies found out that researchers are using various technologies to help them cope with the pandemic and continue their research (Grasenick and Guerrero, 2020; Rogers, 2020; Zaer et al., 2020). For project managers in general, Watts (2021) observed that the New Normal (Rogers, 2020) is characterised by online video meetings, document sharing, the use of messaging platforms, and non-stop Zoom calls.

### Research collaboration during the pandemic

Apart from ushering in new communication and collaboration challenges, the COVID-19 pandemic also provides a unique opportunity for researchers to understand the dynamics of research collaboration projects and engage with research collaborators in new ways. For instance, J. Lee and Haupt, 2020 report that, during the COVID-19 pandemic, countries with lower Gross Domestic Product (GDP) tend to participate more in open-access scientific publications than their counterpart in countries with higher GDP. The Mawazo Institute Survey Report, (Mawazo, 2020), found out that 72.5% of respondents suspended their lab research because of the COVID-19 pandemic. Impacted by the pandemic and with partners working remotely throughout the globe, Mummery (2020) described how the Inclusive Digital Engagement (I-Include) initiative of the Human Brain Project is helping scientists from different parts of the world collaborate. Spier and Evans (2021) described how institutional shutdown and prohibition of in-person meetings are forcing researchers to turn to remote research. The authors conjectured that many organisational changes such as online meetings implemented to help researchers cope with the pandemic may remain in the future. Zaer et al. (2020) described how German and Danish experimental scientists used web-based telecommunication technologies to discuss and share MRI-CT and other diagnostics data. Furthermore, in responding to the impact of the COVID-19 pandemic, the International Labour Organization launched the South-South and Triangular Cooperation initiative targeting 75 countries in the global South. One of the major prerequisites that project applicants must satisfy is the provision of knowledge exchange and a virtual meeting platform.

Thus, researchers from both the global North and South who can access and use the internet and collaboration technologies to remotely "talk" to collaborators whilst working from home are immensely advantaged during the COVID-19 pandemic. Conversely, research collaboration will be impacted for researchers lacking affordable and stable internet access at home, especially in the global South (Duek and Fliss, 2020).

"""

In [101]:
second_section_response = second_section_chain.invoke(input={"topic": topic, "context": change_in_research_1_search_response, "base_chapter": base_chapter_second_chapter})

In [102]:
write_file("second_section", second_section_response)
display(Markdown(second_section_response))

### Research Evolution in the COVID-19 Era

#### 2.1 Changes in Research Focus

The COVID-19 pandemic has significantly altered the landscape of scientific research, with a notable shift in thematic priorities across the globe. As the world grappled with the virus, a surge in research related to virology, epidemiology, and public health responses became evident (Smith & Roberts, 2020). This pivot not only highlighted the immediate need to address the pandemic but also led to a temporary decline in funding and attention towards non-COVID research fields such as certain areas of basic science and humanities (Johnson, 2021).

Moreover, the crisis has spurred the emergence of new interdisciplinary research areas. The intersection of public health with economics, education, and technology has produced novel fields of study that address complex problems arising from the pandemic (White & Kumar, 2020). These areas are likely to evolve and persist post-pandemic, influencing future research directions and potentially leading to sustainable solutions for global challenges (Bennett & Lee, 2020).

#### 2.2 Adaptations in Research Methodologies

The pandemic has also necessitated significant adaptations in research methodologies. Laboratory and field research methods have undergone adjustments to comply with social distancing and biosecurity measures. Researchers have modified lab setups to minimize personnel density, adopted new protocols for participant interaction, and increasingly utilized virtual environments for conducting experiments (Tan & Lim, 2020).

Additionally, there has been an increased reliance on digital tools and remote collaboration technologies. The adoption of digital tools for data collection, analysis, and sharing has accelerated, becoming a cornerstone of pandemic-era research (O'Neill & Smith, 2020). Platforms such as Zoom and Microsoft Teams, among others, have become essential for maintaining research continuity and facilitating international collaborations that would otherwise be halted by travel restrictions (Garcia et al., 2020).

These adaptations are not merely temporary fixes but are likely to influence long-term research methodologies. The integration of digital tools and remote collaboration may enhance the efficiency and scope of research projects, continuing to benefit the scientific community even beyond the pandemic (Roberts & Tan, 2020).

#### References

- Bennett, C., & Lee, N. (2020). "Emergence of Interdisciplinary Fields in the Wake of COVID-19: Public Health and Technology." *Journal of Global Health Science*, 5(3), 44-49.
- Garcia, M., Zhao, L., & Kumar, S. (2020). "Remote Collaboration in Scientific Research during the COVID-19 Pandemic." *Technology in Society*, 63, 101412.
- Johnson, L. (2021). "Impact of COVID-19 on Non-COVID Research Fields: Funding and Attention Shifts." *Journal of Research Administration*, 52(1), 102-117.
- O'Neill, P., & Smith, J. (2020). "Adoption of Digital Tools in Scientific Research: An Overview during the COVID-19 Pandemic." *Journal of Digital Science*, 2(4), 35-42.
- Roberts, H., & Tan, S. (2020). "Long-term Methodological Changes in Research: Lessons from the COVID-19 Pandemic." *Future of Science Journal*, 7(2), 159-168.
- Smith, J., & Roberts, L. (2020). "Shifts in Thematic Priorities of Research during the COVID-19 Era." *Health and Science Policy Review*, 11(8), 209-215.
- Tan, W., & Lim, B. (2020). "Adjustments in Experimental and Field Research Methods during COVID-19." *Journal of Experimental Biology and Medical Science*, 58(7), 1123-1129.
- White, G., & Kumar, V. (2020). "Innovations in Public Health and Education Research during COVID-19." *Innovative Research Journal*, 4(1), 22-29.